In [ ]:
import pandas  
import numpy as np 
from astropy.io import fits 
from sklearn.model_selection import cross_validate
from pyBIA import catalog, ensemble_model

### Create the Data Files to Generate Figure 2 ###

data_path = '/Users/daniel/Desktop/Folders/Lyalpha/pyBIA_Paper_1/data_files/NDWFS_Tiles/Bw_FITS/'
data_error_path = '/Users/daniel/Desktop/Folders/Lyalpha/pyBIA_Paper_1/data_files/NDWFS_Tiles/rms_images/Bw/'
path_to_save = '/Users/daniel/Desktop/BW_TRAINING_SET_TEST/'

#data_path, data_error_path = '/fs1/scratch/godines/NDWFS_Tiles/Bw/', '/fs1/scratch/godines/NDWFS_Tiles/Error_Maps/Bw/'
#path_to_save = '/fs1/scratch/godines/'

training_set = pandas.read_csv('/Users/daniel/Desktop/Folders/pyBIA/pyBIA/data/training_set_objects.csv')
sigs = np.around(np.arange(0.1, 1.51, 0.01), decimals=2)

filt = '_Bw_' #_R_: note that for Red band the data filename ends with 03_reg_fix.fits

for sig in sigs:
    frame = [] #To store all 27 subfields
    for fieldname in np.unique(np.array(training_set['field_name'])):
        # Load the field data
        data, error_map = fits.open(data_path+fieldname+filt+'03_fix.fits'), fits.getdata(data_error_path+fieldname+filt+'03_rms.fits.fz')
        # Extract the data and corresponding ZP
        data_map, zeropoint = data[0].data, data[0].header['MAGZERO']
        # Select only the samples from this subfield
        subfield_index = np.where(training_set['field_name']==fieldname)[0]
        xpix, ypix = training_set[['xpix', 'ypix']].iloc[subfield_index].values.T
        objname, field, flag = training_set[['obj_name', 'field_name', 'flag']].iloc[subfield_index].values.T
        # Create the catalog object
        cat = catalog.Catalog(data_map, error=error_map, x=xpix, y=ypix, zp=zeropoint, nsig=sig, flag=flag, obj_name=objname, field_name=field, invert=True)
        # Generate the catalog and append the ``cat`` attribute to the frame list
        cat.create(save_file=False); frame.append(cat.cat)
    # Combine all 27 sub-catalogs into one master frame and save to the ``path_to_save``
    frame = pandas.concat(frame, axis=0, join='inner'); frame.to_csv(path_to_save+filt+'training_set_nsig_'+str(sig), chunksize=1000)                                                

In [ ]:
# Data for Figure 2

#These are the features to use, note that the catalog includes more than this!
columns = ['mag', 'mag_err', 'm00', 'm10', 'm01', 'm20', 'm11', 'm02', 'm30', 'm21', 'm12', 'm03', 'mu10', 
    'mu01', 'mu20', 'mu11', 'mu02', 'mu30', 'mu21', 'mu12', 'mu03', 'hu1', 'hu2', 'hu3', 'hu4', 'hu5', 'hu6', 'hu7', 
    'legendre_2', 'legendre_3', 'legendre_4', 'legendre_5', 'legendre_6', 'legendre_7', 'legendre_8', 'legendre_9', 
    'area', 'covar_sigx2', 'covar_sigy2', 'covar_sigxy', 'covariance_eigval1', 'covariance_eigval2', 'cxx', 'cxy', 'cyy', 
    'eccentricity', 'ellipticity', 'elongation', 'equivalent_radius', 'fwhm', 'gini', 'orientation', 'perimeter', 
    'semimajor_sigma', 'semiminor_sigma', 'max_value', 'min_value']

rf_scores, xgb_scores = [], [] # To store the accuracies as a function of sigma threshold (Left Panel of Figure 2)
blob_nondetect, other_nondetect = [], [] # To store the number of non-detections (Right Panel of Figure 2)
impute = True; num_cv_folds = 10 # Will impute NaN values and then assess accuracy using 10-fold CV

for sig in sigs:
    # Load each nsig file
    #df = pandas.read_csv('/fs1/scratch/godines/BW_NSIG/training_set_nsig_'+str(sig))
    df = pandas.read_csv(path_to_save+filt+'training_set_nsig_'+str(sig))
    # Omit any non-detections
    mask = np.where((df['area'] != -999) & np.isfinite(df['mag']))[0]
    # Balance both classes to be of same size
    blob_index = np.where(df['flag'].iloc[mask] == 0)[0]
    other_index = np.where(df['flag'].iloc[mask] == 1)[0]
    df_filtered = df.iloc[mask[np.concatenate((blob_index, other_index[:len(blob_index)]))]]
    # Training data arrays
    data_x, data_y = np.array(df_filtered[columns]), np.array(df_filtered['flag'])
    # Create RF model first
    model = ensemble_model.Classifier(data_x, data_y, clf='rf', impute=True); model.create()
    cross_val = cross_validate(model.model, model.data_x, model.data_y, cv=num_cv_folds)
    rf_scores.append(np.mean(cross_val['test_score']))
    # Change to XGB model and re-create
    model.clf = 'xgb'; model.create()
    cross_val = cross_validate(model.model, model.data_x, model.data_y, cv=num_cv_folds)
    xgb_scores.append(np.mean(cross_val['test_score']))
    # This checks how many normalized non-detections occurred at this threshold
    blob_index, other_index = np.where(df['flag'] == 0)[0], np.where(df['flag'] == 1)[0]
    blob_nondetect.append(len(np.where(df.area.iloc[blob_index] == -999)[0]) / len(blob_index))
    other_nondetect.append(len(np.where(df.area.iloc[other_index] == -999)[0]) / len(other_index))

score_data = np.c_[sigs, rf_scores, xgb_scores]
non_detect_data = np.c_[sigs, blob_nondetect, other_nondetect]
#np.savetxt(path_to_save+'nsig_scores'+filt, score_data, header="nsigs, RF_scores, XGB_scores")
#np.savetxt(path_to_save+'non_detections'+filt, non_detect_data, header="nsigs, blob_non_detections, other_non_detections")

In [ ]:
import matplotlib.pyplot as plt   
from matplotlib.ticker import FuncFormatter

### Generate the Plots ###

ensemble_model._set_style_() #The custom matplotlib style

# Figure 2 Left Panel
max_rf_score, max_xgb_score = np.where(score_data[:,1]==np.max(score_data[:,1]))[0], np.where(score_data[:,2]==np.max(score_data[:,2]))[0] 
optimal_index = max_xgb_score[0] if score_data[:,2][max_xgb_score] > score_data[:,1][max_rf_score] else max_rf_score[0]

# ACCURACY PLOT
fig, ax1 = plt.subplots()
lns1, = ax1.plot(score_data[:,0], score_data[:,1], linestyle='--', color='b')
lns2, = ax1.plot(score_data[:,0], score_data[:,2], linestyle='-', color='r')
yscatter = score_data[:,2][optimal_index] if score_data[:,2][max_xgb_score] >= score_data[:,1][max_rf_score] else score_data[:,1][optimal_index]
lns3 = ax1.scatter(score_data[:,0][optimal_index], yscatter, marker='*', s=225, edgecolors='black', c='green', alpha=0.63, label='Optimal')
ax1.legend([lns1, lns2, lns3], ['RF', 'XGBoost', 'Optimal'], loc='upper center', ncol=3, frameon=False, handlelength=2) #r'RF $\pm$ 1$\sigma$'
ax1.set_title('RF vs XGBoost: Baseline Performance')
ax1.set_xlabel(r'$\sigma$ Noise Detection Limit'); ax1.set_ylabel('10-Fold CV Acc')#, size=14)
ax1.set_xlim((0.1, 1.5)); ax1.set_ylim((0.875, 0.93))
plt.savefig('/Users/daniel/Desktop/nsigs.png', dpi=300, bbox_inches='tight')
#plt.show()

# Figure 2 Right Panel

def y_axis_formatter(x, pos):
    return '{:.2f}'.format(round(x, 2))

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
lns1, = ax1.plot(non_detect_data[:,0], non_detect_data[:,2], linestyle='--', color='k')
lns2, = ax2.plot(non_detect_data[:,0], non_detect_data[:,1], linestyle='-', color='k')
lns3 = ax1.scatter(non_detect_data[:,0][optimal_index], non_detect_data[:,2][optimal_index], marker='*', s=225, edgecolors='black', c='green', alpha=0.63, label='Optimal')
ax1.legend([lns1, lns2, lns3], ['OTHER', 'DIFFUSE', 'Optimal'], loc='upper center', ncol=3, frameon=False)
ax1.set_title('Normalized Non-Detections')
ax1.set_xlabel(r'$\sigma$ Noise Detection Limit')
ax2.set_ylabel('DIFFUSE'); ax1.set_ylabel('OTHER')
ax2.set_xlim((0.1, 1.5));ax2.set_ylim((0, 0.16)); ax1.set_ylim(0, 0.7)
ax1.yaxis.set_major_formatter(FuncFormatter(y_axis_formatter))
ax2.yaxis.set_major_formatter(FuncFormatter(y_axis_formatter))
plt.savefig('/Users/daniel/Desktop/nsigs_nondetect.png', dpi=300, bbox_inches='tight')
#plt.show() 